![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## Phidget_7 Game

This is a really open ended exercise for the students.

It is important to run this notebook in the Google Chrome web browser, on a desktop or laptop computer. Opera and Microsoft Edge should work as well. Unfortunately, other browsers (FireFox, Safari) do not work currently, as they cannot access the USB port that connects to the Phidget device.

Reference code: https://www.phidgets.com/education/learn/getting-started-kit-tutorial/blink-led/
Be sure to look at the JavaScript code.

The idea is to create a game using the push buttons and lights on the Phidgets Starter Kit. Some ideas include:
- A speed test, where a light goes on at random and the code measures the time it takes the player to react by pushing the button.
- A two-button speed test. The green or red light comes on at random, and the player must react by hitting the corresponding button. With the speed measured, as above
- A pattern game. The lights flash in a random pattern (e.g. Red, Red, Green, Red, Green) and the player must repeat the pattern by pushing the button. 
- Or perhaps a "prompt" appears on the screen, and the player must push the corresponding button. Say, a picture randomly of an example of "food" or "office supplies" and the player must push "Green" for food, and "Red" for office supplies.

Or think of your own game. 

A good place to start is the code in the notebook Phidget_6_buttons, which shows how to connect to the buttons and turn on and off the light. 

The basic code is repeated here. Can you modify it to make a game?
 

## Setting up the Phidget device

We will be using the Phidget Starter Kit, as shown in this figure:
<img src="images/KIT4003.jpg" alt="Image of the starter kit"  width = 500 />

Verify these points:
- Get a hold of a Phidgets Starter Kit. It should already be assembled. If not, assemble it now.
    - You will have received instructions with the kit on how to assemble it.
- The Starter Kit has a two buttons and two switches connect. If it does not, find them and attach.
    - We assume the red button is attached to port 0, and the red LED attached to port 1.
    - We assume the green button is attached to port 5, and the green LED attached to port 4.
    - if note, either change yoor wires or chage the "setPortHub" values in the code belwo. 
- Connect the Starter Kit to your computer with a USB cable. 
    - The Kit come with a USB cable, it is attached to the Hub device inside the Starter Kit.
- If you have an older version of the Phidgets, you may need to update its firmware. See the following for details:
    - https://www.phidgets.com/education/educators/advanced-troubleshooting/firmware-upgrade/
- As mentioned above, make sure you are running this notebook in the Google Chrome browser. 

## Setup the Phidget device

As in the earlier notebooks, the main steps are thus:

1. Load in some Python libraries
2. Create the PhidgetsButtons class, using Javascript 
3. Display the PhidgetButtons widget
4. Open the USB connection
5. Control the Phidgets hardware from code
6. Link the Phidgets to checkboxes
7. Make a simple game 
8. When we are done, close the device

## Step 1.

Let's import a few Python libraries that we need. 

The **anywidget** library is still in development, so we do a pip install if it is not already on the hub.  

In [1]:
try:
    import anywidget
except:
    !pip install --user "anywidget[dev]"
    import anywidget

## NOTE:

If the **install** above gives an error message. try closing the notebook, shut down the kernel, then reopen the notebook. Run the cell again. After that, the libary will be properly installed.

In [2]:
import traitlets
import threading
import ipywidgets as widgets
from time import sleep, perf_counter

## Step 2.

We create the PhidgetButtons2 class (software object) as an example of the **anywidget** models. It is based on the previous notebook, phidget_6_buttons.ipynb. However, we remove some of the buttons in the user interface, so it is better suited for games. 

The JS code for this tool is shown. Have a read, and feel free to experiment with it. 

In [3]:
class PhidgetButtons2(anywidget.AnyWidget):
    _esm = """

// Importing the functions we need for Phidgets
  import {USBConnection, DigitalInput, DigitalOutput} from "https://esm.sh/phidget22@3.17";

  export function render({ model, el }) {
    
    var connection = 0, inA = 0, inB = 0, outA = 0, outB = 0, connOpen = false;

// Opening the USBconnection
    async function openUSB() {
        if (connection) {
            try {await connection.close();} catch {}
            try {await connection.delete();} catch {}
        }
        try {connection = new USBConnection();} 
            catch{console.log("This browser does not support WebUSB");}

     	try {
                await connection.connect();
                connection.requestWebUSBDeviceAccess();
                connOpen = true;
        } catch(err) {
                connOpen = false;
        }

// Opening the digital inputs (for switched) and outputs (for LEDs)
        inA = new DigitalInput();
        inA.setHubPort(0);        
        inA.setIsHubPortDevice(true);
        inA.onStateChange = function(state) {
            model.set('inAstate', state);
            model.save_changes();
         }

        inB = new DigitalInput();
        inB.setHubPort(5);        
        inB.setIsHubPortDevice(true); 
        inB.onStateChange = function(state) {
            model.set('inBstate', state);
            model.save_changes();
         }

        outA = new DigitalOutput();
        outA.setHubPort(1);        
        outA.setIsHubPortDevice(true); 

        outB = new DigitalOutput();
        outB.setHubPort(4);        
        outB.setIsHubPortDevice(true);
         
        function outAchanged() {
            var state = model.get('outAstate');
            outA.setState(state);
            model.set('outAstate', state);
            model.save_changes();
        }
         
        function outBchanged() {
            var state = model.get('outBstate');
            outB.setState(state);
            model.set('outBstate', state);
            model.save_changes();
        }
         
        model.on("change:outAstate", outAchanged);
        model.on("change:outBstate", outBchanged);


// Here is the code to connect the devices
     	try {
                await inA.open(1000);
                await inB.open(1000);
                await outA.open(1000);
                await outB.open(1000);
                connOpen = true;
        } catch(err) {
                connOpen = false;
        }
       
         // now get the UI to reflect the status
        if (connOpen) {
            textStatus.innerHTML = 'USB connected, channels open.  .';
            button.innerHTML = `Click to disconnect`;
        }
        else {
            textStatus.innerHTML = 'USB did not connect.  .';
            button.innerHTML = `Click to connect`;        
        }
    };

    async function closeUSB() {
        try {await connection.close();} catch {}
        try {await connection.delete();} catch {}
        connOpen = false;
        textStatus.innerHTML = 'Disconnected.  .';
        button.innerHTML = `Click to connect`;
    };
    
// The user interface, with one buttons and text 
    let button = document.createElement("button");
    button.classList.add("ph-button");
    button.innerHTML = `Click to open USB`;
    button.addEventListener("click", async () => {
        if (connOpen) {closeUSB();} else {openUSB();}
    });
    let textStatus = document.createElement("label");
    textStatus.innerHTML = 'Status message here.  .';
    
// Now display the user interface
 
    el.appendChild(button);
    el.appendChild(textStatus);
    
// Clean up when the notebook is closed
    return async () => {
        try {await channel.close();} catch {}
        try {await connection.close();} catch {}
        try {await connection.delete();} catch {}
    };
  }
    """
    _css = """
        .ph-button {color: white; 
            background-color:rgb(96, 107, 174); 
            border-radius: 8px; 
            font-size: 24px; 
            display: block;
            padding: 15px 32px;}
        .ph-button:hover { background-color:rgb(120, 128, 187); }
    """
    inAstate = traitlets.Bool(False).tag(sync=True)
    inBstate = traitlets.Bool(False).tag(sync=True)
    outAstate = traitlets.Bool(False).tag(sync=True)
    outBstate = traitlets.Bool(False).tag(sync=True)

pb = PhidgetButtons2()

## Step 3.

We now open and display the Phidget widget with its button and text. Run the following cell.

In [4]:
display(pb)

PhidgetButtons2()

## Step 4. 
Clicking on the "Connect" button above will open a window that asks you to select the Phidget Hub. Select the hub and click "Connect." The window looks like this:
<img src="images/Pconnect.png" alt="Hey"  width = 300 />

If you don't get this window, perhaps you are using the wrong browser. (Use Chrome, Chromium, or Opera).

If you get the window but don't see the Phidget device, check to be sure the Phidget VINT hub is connected to your computer with a USB cable.

### Check:
If all is well, the status text will report that the USB port is connected and the device channels are open. 

If you don't see this status, check again that the Phidget VINT device is connected to your computer with a USB cable. Also check that you have the LEDs and pushbottom cables connected to the correct ports, as per the setup instruction. 

- We assume the red button (inA) is attached to port 0, and the red LED (outA) attached to port 1.
- We assume the green button (inB) is attached to port 5, and the green LED (outB) attached to port 4.

If these are different on your device, you will need to change these numbers in the JS code above to match the ports on the VINT USB hub. 

You won't see any buttons about the LEDs or buttons. Instead, we use Python code to control these hardware devices. 

## Step 5.

We control the hardware directly with code. Try running the following cells and observe the action.s

In [5]:
# This will turn on the Red LED
pb.outAstate = True

In [6]:
# This will turn off the Red LED
pb.outAstate = False

In [7]:
# This will turn on the Green LED
pb.outBstate = True

In [8]:
# This will turn off the Green LED
pb.outBstate = False

In [9]:
## This will show the state of the Red button. It usually is false
pb.inAstate

False

In [10]:
## Try holding down the Red button, and then run this cell. It will read True
pb.inAstate

False

## Step 6. Linking the phidgets to widgets

We can add some check boxes to monitor the hardware. 

In [11]:
checkinA = widgets.Checkbox(
    value=False,
    description='Button A',
    disabled=False
)
checkinB = widgets.Checkbox(
    value=False,
    description='Button B',
    disabled=False
)
checkoutA = widgets.Checkbox(
    value=False,
    description='LED A',
    disabled=False
)
checkoutB = widgets.Checkbox(
    value=False,
    description='LED B',
    disabled=False
)

linkoutA=widgets.link((checkoutA,'value'),(pb,'outAstate'))
linkoutB=widgets.link((checkoutB,'value'),(pb,'outBstate'))
linkinA=widgets.link((checkinA,'value'),(pb,'inAstate'))
linkinB=widgets.link((checkinB,'value'),(pb,'inBstate'))

In [12]:
display(checkinA,checkinB,checkoutA,checkoutB)

Checkbox(value=False, description='Button A')

Checkbox(value=False, description='Button B')

Checkbox(value=False, description='LED A')

Checkbox(value=False, description='LED B')

### Check.

Push the buttons on the Phidget hardware, and the above checkboxs will active.

Click on the checkboxes for the LEDs, and the physical LED on the Phidget hardware will light up.

## Step 7. Let's try a game.

Let's create game to test  how quickly you can push the button, when the light comes on.

Here is the basic idea for the code:
- time variables t_start and t_end will record when the LED goes on, and when you push the button
- the different (t_end - t_start) tells how long you took. It is displayed in a "text" widget
- the game starts by making sure the LED is out, waits 3 seconds, then turns the LED on
- the game waits up to 3 seconds to see your response. 
- when you push the button, the end time t_end is saved using the function get_time
- the game then ends, and displays your time.

Here is the Python code.

In [13]:
t_start = 0
t_end = 0

def get_time(change):
    global t_end
    if change.new:
        t_end = perf_counter()

pb.observe(get_time, names=["inAstate"])

def run_game():
    global t_start,t_end
    pb.outAstate = False
    sleep(3)
    t_start = perf_counter()
    pb.outAstate = True
    sleep(3)
    pb.outAstate = False
    text.value = f'{(t_end-t_start):.3f} seconds'
    
text = widgets.Text(
    value='0.000 seconds',
    description='Speed:',
    disabled=False   
)
text

Text(value='0.000 seconds', description='Speed:')

## Run it

We run the code in a thread, to make sure the Phidgets and code work together properly.

In [15]:
thread = threading.Thread(target=run_game)

thread.start()

### Check

How did it go? Did you get a time? Run the thread above (previous cell) to try the game again. 

What's your best time?

## Now try your own

Can you build on this example game? Put in a random delay? Have the two LEDs light up at random? Have a pattern of lights that must be matched on the buttons? Get creative?

## Step 8. Closing up

When you are all done, you should close up the Phidget hardware . Click the button to disconnect, or just close this notebook.

## And that's it. 

That was a simple game. Can you build on this idea?


Good luck!

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)